<a href="https://colab.research.google.com/github/tobipe08/OrientEd-sources/blob/main/languageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Cargar el modelo y el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Cargar tus datos (por ejemplo, desde un archivo CSV)
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

# Tokenización de los datos
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',       # Carpeta para guardar resultados
    evaluation_strategy="epoch",  # Evaluar en cada época
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,           # Elige el número de épocas
    weight_decay=0.01,
)

# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train()

results = trainer.evaluate()
print(results)

model.save_pretrained("./model_save")
tokenizer.save_pretrained("./model_save")

#CARGAR EL MODELO
#from transformers import pipeline

# Cargar el modelo ajustado
#classifier = pipeline('text-classification', model='./model_save')

# Usar el modelo para hacer predicciones
#predictions = classifier("¿Qué carrera debería estudiar si me gusta la tecnología?")
#print(predictions)